In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import xgboost as xgb
import pandas as pd
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score
import seaborn as sns

## 500K dataset

In [3]:
data_500 = pd.read_csv("/Users/nirugidla/Documents/GitHub/milestone2_MADS/data_500k.csv", low_memory=False)
data_500

,RECORD_ID,ADD_TYPE,AFAMPROFLS,AGE,AI_COUNTY_NAME,AIRCOND,APP_CHILD,APP_MENBIG,APP_TODDLR,APP_WOMEN,...,VTR_PRI16,VTR_PRI17,VTR_PRI18,VTR_PRI19,VTR_PRI20,VTR_PRI21,VTR_PRI22,WORKWOMAN,YEARBUILT,ZIP
0,403390,S,,21.0,Fairbanks North Star,,,,,,...,,,,,,,,,NaN,99705
1,62285,H,,NaN,Anchorage,,,,,,...,,,,,,,,,NaN,99506
2,331355,,,91.0,Kenai Peninsula,,,,,,...,,,,,,,,,NaN,99603
3,206320,H,,65.0,Anchorage,,,,,,...,,,,,,,Y,,,99567
4,188078,S,,76.0,Juneau,,,,,,...,Y,,Y,,,,Y,Y,1985,99801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,349635,H,,20.0,BIBB,,,,,,...,,,,,,,,,NaN,31204
499996,420654,S,,50.0,COWETA,A,,,,,...,,,,,,,,,2003,30263
499997,131262,S,,19.0,ROCKDALE,,,,,,...,,,,,,,,,,30013
499998,315673,H,,21.0,BARROW,,,,,,...,,,,,,,,,NaN,30680


## Data Cleaning

In [5]:
data_500.sample(n=500)

,RECORD_ID,ADD_TYPE,AFAMPROFLS,AGE,AI_COUNTY_NAME,AIRCOND,APP_CHILD,APP_MENBIG,APP_TODDLR,APP_WOMEN,...,VTR_PRI15,VTR_PRI16,VTR_PRI17,VTR_PRI18,VTR_PRI19,VTR_PRI20,VTR_PRI21,VTR_PRI22,WORKWOMAN,YEARBUILT
84399,496231,S,Unknown,25.0,FLAGLER,A,Unknown,Unknown,Unknown,Y,...,Unknown,Unknown,Unknown,Unknown,Unknown,E,Unknown,Unknown,Unknown,1986
484854,102544,Unknown,Unknown,62.0,TALBOT,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,X,Unknown,1955
226471,373375,H,Unknown,55.0,SCOTT,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
166786,390809,S,Unknown,61.0,SEDGWICK,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
371588,204466,S,Unknown,23.0,NEWPORT,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160451,488765,S,Unknown,39.0,HAMILTON,A,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,D,Unknown,Unknown,Unknown,Y,Unknown,1995
440522,487245,S,Unknown,51.0,PRINCE WILLIAM,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
386269,389513,S,Unknown,37.0,MEIGS,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2000
258849,127079,S,Unknown,42.0,ORANGE,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,M,Unknown,Unknown


In [4]:
# Data Loading
#data_500 = pd.read_csv("/Users/nirugidla/Documents/GitHub/milestone2_MADS/data_500k.csv", low_memory=False)

# Data Cleaning - Column Names
data_500.columns = data_500.columns.str.strip()

# Data Cleaning - Drop ZIP
data_500.drop('ZIP', axis=1, inplace=True)

# Data Cleaning - Drop Duplicates
data_500.drop_duplicates(inplace=True)

# Data Cleaning - Object Columns
for col in data_500.columns:
    if data_500[col].dtype == 'object':
        data_500[col] = data_500[col].str.strip()

# Data Cleaning - Empty Strings
data_500.replace('', 'Unknown', inplace=True)

# Data Cleaning - NaN for Object Types
data_500.loc[:, data_500.dtypes == 'object'] = data_500.loc[:, data_500.dtypes == 'object'].fillna('Unknown')

# Data Cleaning - Drop Columns and Rows with All NaNs
data_500.dropna(axis=1, how='all', inplace=True)
data_500.dropna(axis=0, how='all', inplace=True)

# Identify numeric and non-numeric columns
numeric_cols = data_500.select_dtypes(include=['int64', 'float64']).columns
non_numeric_cols = data_500.select_dtypes(exclude=['int64', 'float64']).columns

# Data Cleaning - Removing Non-Numeric Columns with More Than 90% Missing Data
missing_data_percentage = data_500.isnull().mean() * 100
non_numeric_cols_to_remove = missing_data_percentage[non_numeric_cols]
non_numeric_cols_to_remove = non_numeric_cols_to_remove[non_numeric_cols_to_remove > 90].index.tolist()
data_500_reduced = data_500.drop(columns=non_numeric_cols_to_remove)

# Update the list of non-numeric columns after removal
non_numeric_cols = data_500_reduced.select_dtypes(exclude=['int64', 'float64']).columns

# Identifying Specific Types of Non-Numeric Columns
cols_with_Y_or_Unknown = [col for col in non_numeric_cols if set(data_500_reduced[col].unique()) <= {'Y', 'Unknown'}]
cols_with_more_than_two_categories = [col for col in non_numeric_cols if len(data_500_reduced[col].unique()) > 2]

# Print identified columns
print("Columns with 'Y' or 'Unknown':", cols_with_Y_or_Unknown, len(cols_with_Y_or_Unknown))
print("Columns with more than two categories:", cols_with_more_than_two_categories, len(cols_with_more_than_two_categories))

Columns with 'Y' or 'Unknown': ['AFAMPROFLS', 'APP_CHILD', 'APP_MENBIG', 'APP_TODDLR', 'APP_WOMEN', 'APP_WOMPET', 'APP_WOMPLS', 'APP_YNGMEN', 'ARTS', 'AUTOACCES', 'AUTOWORK', 'BOATING', 'BROADERLIV', 'CARDUSER', 'CATOWNER', 'CH_0002FEM', 'CH_0002MAL', 'CH_0002UNK', 'CH_0305FEM', 'CH_0305MAL', 'CH_0305UNK', 'CH_0610FEM', 'CH_0610MAL', 'CH_0610UNK', 'CH_1115FEM', 'CH_1115MAL', 'CH_1115UNK', 'CH_1617FEM', 'CH_1617MAL', 'CH_1617UNK', 'CHRISTFAM', 'COL_ANTIQ', 'COL_ARTS', 'COL_COIN', 'COL_SPORT', 'COL_STAMP', 'COMPHOMOFC', 'COMPUTERS', 'COOK_GEN', 'CURRAFFAIR', 'DEPTSTCRD', 'DIETING', 'DIYLIV', 'DOGOWNER', 'DON_ANML', 'DON_ARTCUL', 'DON_CHARIT', 'DON_CHILD', 'DON_ENVIR', 'DON_ENVWLD', 'DON_HEALTH', 'DON_INTAID', 'DON_OTHER', 'DON_POLCONS', 'DON_POLIT', 'DON_POLLIB', 'DON_RELIG', 'DON_VET', 'DONATION', 'EDU_ONLINE', 'EQUESTRIAN', 'EXER_GROUP', 'GAMING', 'GARDENER', 'GOLF', 'GRANDCHLD', 'HEALTHBEAU', 'HEATHMED', 'HH_SENIOR', 'HH_VETERAN', 'HH_YOUNGAD', 'HIGHBROW', 'HIGHENDAPP', 'HISTMIL', 'HI

## Just the columns with Yes or Unknown non_numeric_cols 

In [8]:
data_reduced_with_Y_or_Unknown = data_500[cols_with_Y_or_Unknown].copy()
data_reduced_with_Y_or_Unknown.shape

(500000, 141)

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assuming cols_with_Y_or_Unknown has been identified and data_500 has been cleaned
data_reduced_with_Y_or_Unknown = data_500[cols_with_Y_or_Unknown].copy()

# Adding the target column to this data
data_reduced_with_Y_or_Unknown['PARTY_CODE'] = data_500['PARTY_CODE']

# Remove rows where 'PARTY_CODE' is missing, as it's our target variable
data_reduced_with_Y_or_Unknown = data_reduced_with_Y_or_Unknown[data_reduced_with_Y_or_Unknown['PARTY_CODE'].notna()]

# Sample 100,000 rows from the data
data_sample = data_reduced_with_Y_or_Unknown.sample(n=100000, random_state=42)

# Count instances of each class in 'PARTY_CODE' again to filter out classes with fewer than 2 instances
class_counts = data_sample['PARTY_CODE'].value_counts()
valid_classes = class_counts[class_counts >= 2].index.tolist()
data_sample = data_sample[data_sample['PARTY_CODE'].isin(valid_classes)]

# Label-encode 'PARTY_CODE' column
le = LabelEncoder()
data_sample['PARTY_CODE'] = le.fit_transform(data_sample['PARTY_CODE'].astype(str))

# One-Hot Encoding
data_one_hot = pd.get_dummies(data_sample, columns=cols_with_Y_or_Unknown, drop_first=True)

# Splitting the Data into Training and Test Sets
X = data_one_hot.drop('PARTY_CODE', axis=1)
y = data_one_hot['PARTY_CODE']

# Perform train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and Train XGBoost Classifier
xgb = XGBClassifier(objective='multi:softmax', random_state=42)
xgb.fit(X_train, y_train)

# Make Predictions and Evaluate the Model
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"XGBoost Model Accuracy: {accuracy}")

XGBoost Model Accuracy: 0.63215


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder

def run_xgboost(sample_size, num_runs, top_N_features):
    results_list = []

    for run in range(1, num_runs + 1):
        # Assuming cols_with_Y_or_Unknown has been identified and data_500 has been cleaned
        data_reduced_with_Y_or_Unknown = data_500[cols_with_Y_or_Unknown].copy()

        # Adding the target column to this data
        data_reduced_with_Y_or_Unknown['PARTY_CODE'] = data_500['PARTY_CODE']

        # Remove rows where 'PARTY_CODE' is missing, as it's our target variable
        data_reduced_with_Y_or_Unknown = data_reduced_with_Y_or_Unknown[data_reduced_with_Y_or_Unknown['PARTY_CODE'].notna()]

        # Sample data
        data_sample = data_reduced_with_Y_or_Unknown.sample(n=sample_size, random_state=run)

        # Dynamic Class Handling
        class_counts = data_sample['PARTY_CODE'].value_counts()
        valid_classes = class_counts[class_counts >= 2].index.tolist()
        data_sample = data_sample[data_sample['PARTY_CODE'].isin(valid_classes)]

        # Label-encode 'PARTY_CODE' column
        le = LabelEncoder()
        data_sample['PARTY_CODE'] = le.fit_transform(data_sample['PARTY_CODE'].astype(str))

        # One-Hot Encoding
        data_one_hot = pd.get_dummies(data_sample, columns=cols_with_Y_or_Unknown, drop_first=True)

        # Splitting the Data into Training and Test Sets
        X = data_one_hot.drop('PARTY_CODE', axis=1)
        y = data_one_hot['PARTY_CODE']

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=run, stratify=y
        )

        # Initialize and Train XGBoost Classifier
        xgb = XGBClassifier(objective='multi:softmax', random_state=run)
        xgb.fit(X_train, y_train)

        # Make Predictions and Evaluate the Model
        y_pred = xgb.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')

        # Get top N features
        feature_importances = xgb.feature_importances_
        sorted_idx = feature_importances.argsort()[::-1][:top_N_features]
        top_features = X.columns[sorted_idx].tolist()

        # Append to results list
        results_list.append({
            'Run': run,
            'Accuracy': accuracy,
            'F1_Score': f1,
            'Recall': recall,
            'Top_N_Features': top_features
        })

    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results_list)
    
    return results_df

# Example of how to call this function
result = run_xgboost(sample_size=10000, num_runs=10, top_N_features=10)
result

,Run,Accuracy,F1_Score,Recall,Top_N_Features
0,1,0.5965,0.580370,0.5965,"[PRFL_BIDEN_SUPPORT_Y, PRFL_BORDER_SECURITY_Y,..."
1,2,0.6105,0.595713,0.6105,"[PRFL_LIBERAL_NEWS_Y, PRFL_BIDEN_SUPPORT_Y, PR..."
2,3,0.6290,0.610276,0.6290,"[PRFL_BIDEN_SUPPORT_Y, PRFL_LIBERAL_NEWS_Y, PR..."
3,4,0.6205,0.604047,0.6205,"[PRFL_BIDEN_SUPPORT_Y, PRFL_LIBERAL_NEWS_Y, PR..."
4,5,0.6140,0.597579,0.6140,"[PRFL_LIBERAL_NEWS_Y, PRFL_BIDEN_SUPPORT_Y, PR..."
5,6,0.6205,0.604001,0.6205,"[PRFL_BIDEN_SUPPORT_Y, PRFL_LIBERAL_NEWS_Y, PR..."
6,7,0.6160,0.600376,0.6160,"[PRFL_BIDEN_SUPPORT_Y, PRFL_LIBERAL_NEWS_Y, PR..."
7,8,0.6150,0.600305,0.6150,"[PRFL_LIBERAL_NEWS_Y, PRFL_BIDEN_SUPPORT_Y, PR..."
8,9,0.6230,0.608591,0.6230,"[PRFL_LIBERAL_NEWS_Y, PRFL_BIDEN_SUPPORT_Y, PR..."
9,10,0.6220,0.603847,0.6220,"[PRFL_BIDEN_SUPPORT_Y, PRFL_LIBERAL_NEWS_Y, PR..."


In [22]:
for run in result['Top_N_Features']:
    print(run)
    print("+++++")

['PRFL_LIBERAL_NEWS_Y', 'PRFL_BIDEN_SUPPORT_Y', 'PRFL_BORDER_SECURITY_Y', 'PRFL_CONSERVATIVE_NEWS_Y', 'PRFL_MARIJUANA_REFORM_Y', 'PRFL_SANDERS_SUPPORT_Y', 'PRFL_2NDAMEND_Y', 'PRFL_IMMIGRATION_REFORM_Y', 'PRFL_FENCE_SITTER_Y', 'PRFL_TRUMP_SUPPORT_Y']
+++++
['PRFL_LIBERAL_NEWS_Y', 'PRFL_BIDEN_SUPPORT_Y', 'PRFL_BORDER_SECURITY_Y', 'PRFL_CONSERVATIVE_NEWS_Y', 'PRFL_MARIJUANA_REFORM_Y', 'PRFL_SANDERS_SUPPORT_Y', 'PRFL_2NDAMEND_Y', 'PRFL_IMMIGRATION_REFORM_Y', 'PRFL_FENCE_SITTER_Y', 'PRFL_TRUMP_SUPPORT_Y']
+++++


In [24]:
from collections import Counter

# Assuming result is your DataFrame and 'Top_N_Features' is the column with the lists of top features
all_features = [feature for sublist in result['Top_N_Features'].tolist() for feature in sublist]

# Count the frequency of each feature
feature_counts = Counter(all_features)

# Find the most unique features (those that appear only once across all runs)
most_unique_features = [feature for feature, count in feature_counts.items() if count == 1]

# Find the most common features (those that appear the most across all runs)
most_common_features = [feature for feature, count in feature_counts.most_common()]

print("Most Unique Features:", most_unique_features)
print("Most Common Features:", most_common_features)

Most Unique Features: ['PRFL_FENCE_SITTER_Y', 'GAMING_Y', 'DIYLIV_Y', 'RELIGINSP_Y', 'PRFL_BLM_SUPPORT_Y', 'HUNTING_Y', 'SPEC_HOCK_Y', 'CH_1617FEM_Y', 'HUNTSHOOT_Y']
Most Common Features: ['PRFL_BIDEN_SUPPORT_Y', 'PRFL_BORDER_SECURITY_Y', 'PRFL_LIBERAL_NEWS_Y', 'PRFL_CONSERVATIVE_NEWS_Y', 'PRFL_MARIJUANA_REFORM_Y', 'PRFL_SANDERS_SUPPORT_Y', 'PRFL_2NDAMEND_Y', 'PRFL_IMMIGRATION_REFORM_Y', 'PRFL_TRUMP_SUPPORT_Y', 'CHRISTFAM_Y', 'CH_1617MAL_Y', 'PRFL_FENCE_SITTER_Y', 'GAMING_Y', 'DIYLIV_Y', 'RELIGINSP_Y', 'PRFL_BLM_SUPPORT_Y', 'HUNTING_Y', 'SPEC_HOCK_Y', 'CH_1617FEM_Y', 'HUNTSHOOT_Y']


## Just the non_numeric_cols with more than 2 values

In [5]:
data_reduced_with_more_than_two_categories = data_500[cols_with_more_than_two_categories].copy()
data_reduced_with_more_than_two_categories.shape

(500000, 125)

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assuming cols_with_Y_or_Unknown has been identified and data_500 has been cleaned
data_reduced_with_more_than_two_categories = data_500[cols_with_more_than_two_categories].copy()

# Adding the target column to this data
data_reduced_with_more_than_two_categories['PARTY_CODE'] = data_500['PARTY_CODE']

# Remove rows where 'PARTY_CODE' is missing, as it's our target variable
data_reduced_with_more_than_two_categories = data_reduced_with_more_than_two_categories[data_reduced_with_more_than_two_categories['PARTY_CODE'].notna()]

# Sample 100,000 rows from the data
data_sample = data_reduced_with_more_than_two_categories.sample(n=100000, random_state=42)

# Count instances of each class in 'PARTY_CODE' again to filter out classes with fewer than 2 instances
class_counts = data_sample['PARTY_CODE'].value_counts()
valid_classes = class_counts[class_counts >= 2].index.tolist()
data_sample = data_sample[data_sample['PARTY_CODE'].isin(valid_classes)]

# Label-encode 'PARTY_CODE' column
le = LabelEncoder()
data_sample['PARTY_CODE'] = le.fit_transform(data_sample['PARTY_CODE'].astype(str))
print("Columns after label encoding:", data_sample.columns)


# Identify the columns in `cols_with_more_than_two_categories` that are actually present in `data_sample`
# Identify the columns in `cols_with_more_than_two_categories` that are actually present in `data_sample`
cols_to_encode = [col for col in cols_with_more_than_two_categories if col in data_sample.columns and col != 'PARTY_CODE']

# One-Hot Encoding
data_one_hot = pd.get_dummies(data_sample, columns=cols_to_encode, drop_first=True)
#print("Columns after one-hot encoding:", data_one_hot.columns)
#print("Columns before dropping PARTY_CODE:", data_one_hot.columns)

# Splitting the Data into Training and Test Sets
X = data_one_hot.drop('PARTY_CODE', axis=1)
y = data_one_hot['PARTY_CODE']

# Perform train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Sanitize column names to remove characters not allowed by XGBoost
X_train.columns = X_train.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')
X_test.columns = X_test.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')

# Initialize and Train XGBoost Classifier
xgb = XGBClassifier(objective='multi:softmax', random_state=42)
xgb.fit(X_train, y_train)

# Make Predictions and Evaluate the Model
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"XGBoost Model Accuracy: {accuracy}")

Columns after label encoding: Index(['ADD_TYPE', 'AI_COUNTY_NAME', 'AIRCOND', 'ASSMLCODE', 'BUS_OWNER',
       'CENSUS_ST', 'CNS_MEDINC', 'CONG_DIST', 'COUNTY_ST', 'COUNTY_TYPE',
       ...
       'VTR_PRI14', 'VTR_PRI15', 'VTR_PRI16', 'VTR_PRI17', 'VTR_PRI18',
       'VTR_PRI19', 'VTR_PRI20', 'VTR_PRI21', 'VTR_PRI22', 'YEARBUILT'],
      dtype='object', length=125)
XGBoost Model Accuracy: 0.932


In [20]:
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import pandas as pd

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Elapsed time: {elapsed_time:.2f} seconds")
        return result
    return wrapper

@timer_decorator
def run_xgboost(data, cols_with_more_than_two_categories, sample_size=100000, num_runs=1, top_N_features=10):
    results_list = []

    for run in range(1, num_runs + 1):
        print(f"Starting run {run}...")
        # Replicate the original code inside the loop
        data_reduced_with_more_than_two_categories = data[cols_with_more_than_two_categories].copy()
        data_reduced_with_more_than_two_categories['PARTY_CODE'] = data['PARTY_CODE']
        data_reduced_with_more_than_two_categories = data_reduced_with_more_than_two_categories[data_reduced_with_more_than_two_categories['PARTY_CODE'].notna()]
        data_sample = data_reduced_with_more_than_two_categories.sample(n=sample_size, random_state=42)
        
        class_counts = data_sample['PARTY_CODE'].value_counts()
        valid_classes = class_counts[class_counts >= 2].index.tolist()
        data_sample = data_sample[data_sample['PARTY_CODE'].isin(valid_classes)]
        
        le = LabelEncoder()
        data_sample['PARTY_CODE'] = le.fit_transform(data_sample['PARTY_CODE'].astype(str))
        
        cols_to_encode = [col for col in cols_with_more_than_two_categories if col in data_sample.columns and col != 'PARTY_CODE']
        data_one_hot = pd.get_dummies(data_sample, columns=cols_to_encode, drop_first=True)
        
        X = data_one_hot.drop('PARTY_CODE', axis=1)
        y = data_one_hot['PARTY_CODE']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        X_train.columns = X_train.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')
        X_test.columns = X_test.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')
        
        xgb = XGBClassifier(objective='multi:softmax', random_state=42)
        xgb.fit(X_train, y_train)

        y_pred = xgb.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')

        feature_importances = xgb.feature_importances_
        sorted_idx = feature_importances.argsort()[::-1][:top_N_features]
        top_features = X.columns[sorted_idx].tolist()

        results_list.append({
            'Run': run,
            'Accuracy': accuracy,
            'F1_Score': f1,
            'Recall': recall,
            'Top_N_Features': top_features
        })

    results_df = pd.DataFrame(results_list)
    
    return results_df

# Assuming data_500 and cols_with_more_than_two_categories are already defined
result = run_xgboost(data_500, cols_with_more_than_two_categories, sample_size=50000, num_runs=2, top_N_features=10)


Starting run 1...
Starting run 2...
Elapsed time: 321.73 seconds


In [18]:
#result['Top_N_Features'][0]
result

,Run,Accuracy,F1_Score,Recall,Top_N_Features
0,1,0.93566,0.928801,0.93566,"[CENSUS_ST_48, PRFL_POLITICAL_IDEOLOGY_L, CENS..."
1,2,0.93566,0.928801,0.93566,"[CENSUS_ST_48, PRFL_POLITICAL_IDEOLOGY_L, CENS..."


In [19]:
from collections import Counter

# Assuming result is your DataFrame and 'Top_N_Features' is the column with the lists of top features
all_features = [feature for sublist in result['Top_N_Features'].tolist() for feature in sublist]

# Count the frequency of each feature
feature_counts = Counter(all_features)

# Find the most unique features (those that appear only once across all runs)
most_unique_features = [feature for feature, count in feature_counts.items() if count == 1]

# Find the most common features (those that appear the most across all runs)
most_common_features = [feature for feature, count in feature_counts.most_common()]

print("Most Unique Features:", most_unique_features)
print("Most Common Features:", most_common_features)

Most Unique Features: []
Most Common Features: ['CENSUS_ST_48', 'PRFL_POLITICAL_IDEOLOGY_L', 'CENSUS_ST_13', 'PRFL_POLITICAL_IDEOLOGY_Unknown', 'TOD_PRES_DIFF_2020_PREC_Unknown']
